In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

In [3]:
import torch.optim as optim
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = nn.functional.relu(out)
        out = nn.functional.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out)#10
        out = nn.functional.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = nn.functional.relu(out)
        out = self.fc2(out)
        out = nn.functional.log_softmax(out,dim=1)
        return out

model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    #if(batch_idx+1)%30 == 0: 
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
# test and train
for epoch in range(1, NUM_EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)
# evaluate
# TODO:calculate the accuracy using traning and testing dataset

Train Epoch: 1 [59776/60000 (100%)]	Loss: 0.058847

Test set: Average loss: 0.0624, Accuracy: 9770/10000 (98%)

Train Epoch: 2 [59776/60000 (100%)]	Loss: 0.057797

Test set: Average loss: 0.0352, Accuracy: 9871/10000 (99%)

Train Epoch: 3 [59776/60000 (100%)]	Loss: 0.028039

Test set: Average loss: 0.0350, Accuracy: 9877/10000 (99%)

Train Epoch: 4 [59776/60000 (100%)]	Loss: 0.014067

Test set: Average loss: 0.0391, Accuracy: 9856/10000 (99%)

Train Epoch: 5 [59776/60000 (100%)]	Loss: 0.011943

Test set: Average loss: 0.0337, Accuracy: 9876/10000 (99%)

Train Epoch: 6 [59776/60000 (100%)]	Loss: 0.003886

Test set: Average loss: 0.0319, Accuracy: 9884/10000 (99%)

Train Epoch: 7 [59776/60000 (100%)]	Loss: 0.006936

Test set: Average loss: 0.0324, Accuracy: 9888/10000 (99%)

Train Epoch: 8 [59776/60000 (100%)]	Loss: 0.001616

Test set: Average loss: 0.0358, Accuracy: 9887/10000 (99%)

Train Epoch: 9 [59776/60000 (100%)]	Loss: 0.003711

Test set: Average loss: 0.0340, Accuracy: 9887/10000